<a href="https://colab.research.google.com/github/vahid-nejad/OCR-Correction/blob/main/ocr2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.4 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.3 MB/s eta 0:00:00


In [ ]:
MAX_LENGTH = 128

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/ds/read_res.csv")
print(df)

                       image  \
0         {011C011C1},{594}:   
1        {011C011C90},{821}:   
2        {011C011C90},{851}:   
3        {011C011C90},{978}:   
4        {011C011C90},{972}:   
...                      ...   
4479   {111TB111TB18},{306}:   
4480   {111TB111TB18},{628}:   
4481   {111TB111TB18},{873}:   
4482   {111TB111TB18},{955}:   
4483  {111TB111TB18},{1434}:   

                                            groundtruth  \
0     \nخدمات مهندسی، ساخت و تولید، اجرا، تامین، ساخ...   
1                                     \nکوکو سبزی\n\t\t   
2                                \nکوکو سیب زمینی\n\t\t   
3                                         \nفلافل\n\t\t   
4                              \nلازانیای سبزیجات\n\t\t   
...                                                 ...   
4479  \nسپس، با خارج نمودن آن از روی تختۀ چوبی، با م...   
4480  \nفعالیت: یک حیوان (اعم از چارپا و یا پرنده) ر...   
4481  \nفکر کنید: دست/u200cهای شخصیت سه/u200cبعدی صح...   
4482  \nهمان/u200cگونه که

In [ ]:
df= df.dropna(subset=['easyocr'])

In [ ]:
selected_column = df['easyocr']
mispelled_texts = selected_column.tolist()


In [ ]:

selected_column = df['groundtruth']
corrected_texts = selected_column.tolist()

In [ ]:
import torch
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, AdamW
from torch.utils.data import DataLoader, Dataset

class CorrectionDataset(Dataset):
    def __init__(self, mispelled_texts, corrected_texts):
        self.mispelled_texts = mispelled_texts
        self.corrected_texts = corrected_texts
        self.tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")

    def __len__(self):
        return len(self.mispelled_texts)

    def __getitem__(self, idx):
        source_text = self.mispelled_texts[idx]
        target_text = self.corrected_texts[idx]

        source_tokens = self.tokenizer.encode(source_text, return_tensors='pt', padding='max_length',max_length=MAX_LENGTH, truncation=True)
        target_tokens = self.tokenizer.encode(target_text, return_tensors='pt', padding='max_length',max_length=MAX_LENGTH, truncation=True)

        return {
            "input_ids": source_tokens.squeeze(),
            "labels": target_tokens.squeeze()
        }


correction_dataset = CorrectionDataset(mispelled_texts, corrected_texts)
correction_dataloader = DataLoader(correction_dataset, batch_size=8, shuffle=True)

In [ ]:
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")


In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-4)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_batches = len(correction_dataloader)
    index = 0
    for  batch in correction_dataloader:
        index+=1
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()

        outputs = model(input_ids=batch["input_ids"], labels=batch["labels"])
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if index % 100==0:
            print(f"batch:{index}/{total_batches} , epoch:{epoch}")


    average_loss = total_loss / len(correction_dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {average_loss:.4f}")


batch:100/479 , epoch:0
batch:200/479 , epoch:0
batch:300/479 , epoch:0
batch:400/479 , epoch:0
Epoch 1/3, Average Loss: 4.7519
batch:100/479 , epoch:1
batch:200/479 , epoch:1
batch:300/479 , epoch:1
batch:400/479 , epoch:1
Epoch 2/3, Average Loss: 3.1678
batch:100/479 , epoch:2
batch:200/479 , epoch:2
batch:300/479 , epoch:2
batch:400/479 , epoch:2
Epoch 3/3, Average Loss: 2.7899


In [ ]:
model.save("YOUR_PATH_TO_SAVE_THE_MODEL")